In [1]:
!pip install -q transformers torch torchvision pillow


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import re
from google.colab import files
import matplotlib.pyplot as plt
from google.colab import files


In [ ]:
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print("Uploaded:", image_path)

img = Image.open(image_path).convert("RGB")
plt.imshow(img)
plt.axis("off")
plt.show()


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

print("BLIP model loaded on", device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

BLIP model loaded on cuda


In [9]:
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        caption_ids = model.generate(**inputs, max_new_tokens=40)
    caption = processor.decode(caption_ids[0], skip_special_tokens=True)
    return caption

caption = generate_caption(image_path)
print("Image Description:\n", caption)


Image Description:
 there is a pineapple and tomatoes on the table royalty photo


In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")  # small English NLP model

def extract_ingredients_from_caption(caption):
    doc = nlp(caption)
    # Extract nouns and adjectives (likely ingredients)
    candidates = [token.text.lower() for token in doc if token.pos_ in ["NOUN", "PROPN", "ADJ"]]
    # Basic cleaning (remove duplicates, common non-food words)
    blacklist = {"plate","bowl","table","kitchen","meal","food","vegetable","fruit","royalty","photo","ingrident","bottle"}
    ingredients = [w for w in candidates if w not in blacklist and len(w) > 2]
    # Remove duplicates
    ingredients = list(dict.fromkeys(ingredients))
    return ingredients

ingredients = extract_ingredients_from_caption(caption)
print("Detected Ingredients:\n", ingredients)


NameError: name 'caption' is not defined